In [ ]:
import pandas as pd
import nltk
from google.colab import files
import numpy as np
import spacy
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text #para hacer stop_word 

nltk.download('stopwords')
from nltk.corpus import stopwords

#Spacy spanish piple
!python -m spacy download es_core_news_lg

Mount your dataset (In my case, i have it in drive)

In [ ]:

from google.colab import drive
drive.mount("/content/drive")



Mounted at /content/drive


In [ ]:
dataset=pd.read_csv('/content/drive/MyDrive/Datasets/dataset.csv')
print(dataset.value_counts('class'))
dataset.sample(10)

class
suicida       43381
no_suicida    40735
dtype: int64


,traducido,class
81367,¿Pensaste que olvidé el tiempo de apreciación ...,no_suicida
72985,Así que necesito ayuda de las personas de Inte...,no_suicida
30443,"Un poco de ventilación, intentaré no hacer que...",suicida
18464,Qué doloroso es cortar considerando cortarme l...,suicida
54633,"Ah sí, mi signo del zodiaco favorito, Virgin A...",no_suicida
26882,"Pensando en ir a Tonighthi impacientes allí, c...",suicida
29276,"El mejor amigo no tiene apoyo de sus hermanos,...",suicida
29160,"30 Dayhi, soy alguien que ha estado contemplan...",suicida
67601,Las redes sociales apestan a las personas que ...,no_suicida
60817,"Voy a ser contundente, pero ¿alguien quiere ha...",no_suicida


<h1>Clean up texts <h1>

In [ ]:
dataset = dataset.drop_duplicates('traducido')
dataset=dataset.dropna()

def clean_text(text):
    # convert text to lowercase
    text = text.lower()

    #Remove URLs
    text=re.sub(r'http:?\S+','',text)

    # Remove punctuation characters
    text = re.sub("[%s]" % re.escape(string.punctuation), " ", text)
    
    # Remove non-ASCII characters, but ncluding Latin characters
    text = re.sub("([^\x00-\x7F\u00C0-\u017F])+", " ", text)
  

    #remove extra spaces and words with less than 2 characters
    filter = [palabra for palabra in text.split() if (len(palabra) > 2 or palabra=='no') and palabra.isalpha() ]
    text = " ".join(filter)
   
    return text



An example clean up text

In [ ]:
clean_text("HOla      приве   comer/cocinar   estás es no 2000 u bebé, s.?#https:\swww.facebook.com niño")


'hola comer cocinar estás no bebé niño'

In [ ]:
dataset['text_clean'] = dataset['traducido'].apply(lambda x: clean_text(x))

<h1>StopWords<h1>

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop=list(stopwords.words("spanish"))
stop.extend(['pok	','megathiccc','snoo','fillerfiller','relleno', 'rellenador','eve','filler','maíz','pop','aaaaaaaaaaaaaaaaaaaaaaaaa','pog','nnn','redader','nnn','nowoww','mei','iti','nowlow','upi'])

def remove_stopwords(text):
    stopwords_esp = stopwords.words('spanish') 
    words = text.split()
    text= [word for word in words if word not in stopwords_esp]
    return " ".join(text)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
dataset['text_clean_stop']=dataset['text_clean'].apply(lambda x: remove_stopwords(x))

without_stop= dataset['traducido'].str.len().sum()
with_stop= dataset['text_clean'].str.len().sum()
print("total characters with stopwords and without clean up %s, with cleaning and without stopwords %s" %(without_stop,with_stop))
      
print("Remove --> ", (1-(with_stop/without_stop))*100,"%")




total characters with stopwords and without clean up 53551884, with cleaning and without stopwords 45512233
Remove -->  15.012825692556397 %


<h1>Lemmatization<h1>

In [ ]:
# load pipeline spanish
nlp = spacy.load('es_core_news_lg')


def lemmatization(text):
  doc = nlp(text)
  lemas_fila = [token.lemma_ for token in doc]
  text=' '.join(lemas_fila)
  return ' '.join(lemas_fila)

In [ ]:
dataset['text_lemma']=dataset['text_clean'].apply(lambda x: lemmatization(x))


<h1>Tokenize texts<h1>

In [41]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')
def tokenize(text):
  words = text.split()
  stopwords_esp = stopwords.words('spanish') 
  return [word for word in nltk.word_tokenize(text) if word.lower() not in stop]



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [44]:
dataset['text_tokenize']=dataset['text_lemma'].apply(lambda x: tokenize(x))

<h1>Vectorize text<h1>

In [ ]:
#min_df=0.001  max_df=90    
#Note1: it´s better to use bigram than unigram, but needs more processing power, min_df and max_df.
#Note2: To find a min and a max, you must try several values. Recommendation max_df(0.7-0.9)
vectorizer=TfidfVectorizer(ngram_range=(1,1),min_df=0.001,max_df=0.9)  
vectors=vectorizer.fit_transform(dataset.text_lemma)    
vectorTf = vectorizer.transform(dataset.text_lemma).toarray()

print(vectorTf.shape)                                               

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['pok'] not in stop_words.
  warnings.warn(


(84114, 5105)


In [ ]:
vectorizer.get_feature_names_out()[:50]

array(['abajo', 'abandonado', 'abandonar', 'abandono', 'abdominal',
       'abeja', 'abiertamente', 'abierto', 'abismo', 'abofetear',
       'abogado', 'abordar', 'abortar', 'aborto', 'abrazar', 'abrazo',
       'abril', 'abrir', 'abrumado', 'abrumador', 'abrumar',
       'absolutamente', 'absoluto', 'absorber', 'absurdo', 'abuela',
       'abuelo', 'aburrido', 'aburrimiento', 'aburrir', 'abusado',
       'abusador', 'abusar', 'abusivo', 'abuso', 'acabar', 'academia',
       'académicamente', 'académico', 'acantilado', 'acariciar', 'acaso',
       'acceder', 'accesible', 'acceso', 'accidental', 'accidentalmente',
       'accidente', 'acción', 'acechar'], dtype=object)

<h1>Result<h1>

In [46]:
dataset.sample(10)

,traducido,class,text_clean,text_clean_stop,text_lemma,text_tokenize
58792,Cualquiera que quiere darme algunos rasguños d...,no_suicida,cualquiera que quiere darme algunos rasguños c...,cualquiera quiere darme rasguños cabeza pelo r...,cualquiera que querer dar yo alguno rasguño ca...,"[cualquiera, querer, dar, alguno, rasguño, cab..."
20348,A veces quiero morir ... Nunca he publicado aq...,suicida,veces quiero morir nunca publicado aquí antes ...,veces quiero morir nunca publicado aquí pasand...,vez querer morir nunca publicar aquí antes no ...,"[vez, querer, morir, nunca, publicar, aquí, pa..."
78784,"Sabes lo que obtuve para Navidad, obtuve liter...",no_suicida,sabes que obtuve para navidad obtuve literalme...,sabes obtuve navidad obtuve literalmente ram n...,saber que obtener para navidad obtener literal...,"[saber, obtener, navidad, obtener, literalment..."
15945,¿Es extraño que pueda tener relaciones sexuale...,suicida,extraño que pueda tener relaciones sexuales si...,extraño pueda tener relaciones sexuales emocio...,extraño que poder tener relación sexual sin em...,"[extraño, poder, tener, relación, sexual, emoc..."
41651,"10 componentes de la última nota efectiva, ¿qu...",suicida,componentes última nota efectiva qué encontrar...,componentes última nota efectiva encontraría ú...,componente último nota efectivo qué encontrar ...,"[componente, último, nota, efectivo, encontrar..."
81643,"Lo he hecho, comí un plato entero de fideos co...",no_suicida,hecho comí plato entero fideos con salsa tomat...,hecho comí plato entero fideos salsa tomate mi...,hecho comer plato entero fideo con salsa tomat...,"[hecho, comer, plato, entero, fideo, salsa, to..."
64209,Si quieres deshacerte de los anuncios en Spoti...,no_suicida,quieres deshacerte los anuncios spotify cierre...,quieres deshacerte anuncios spotify cierre apl...,querer deshacerte el anuncio spotify cerrar ap...,"[querer, deshacerte, anuncio, spotify, cerrar,..."
40014,Justo por qué ... ¿Por qué mi colaboración te ...,suicida,justo por qué por qué colaboración dio collar ...,justo colaboración dio collar dio novio nuevo ...,justo por qué por qué colaboración dar collar ...,"[justo, colaboración, dar, collar, dar, novio,..."
39481,"Me extendí y nadie caresyes, sé que todos tien...",suicida,extendí nadie caresyes que todos tienen sus pr...,extendí nadie caresyes propios problemas pasan...,extendí nadie caresy que todo tener su propio ...,"[extendí, nadie, caresy, tener, propio, proble..."
3296,El adolescente disfórico de género suicida se ...,suicida,adolescente disfórico género suicida queja por...,adolescente disfórico género suicida queja min...,adolescente disfórico género suicida queja por...,"[adolescente, disfórico, género, suicida, quej..."


<h1>Download and use<h1>

In [ ]:
data_clean=dataset[['class','text_clean'] ]

data_clean.to_csv("dataset_clean.csv", index=False)
files.download('dataset_clean.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
data_stop=dataset[['class','text_clean_stop'] ]

data_stop.to_csv("text_clean_stop.csv", index=False)
files.download('text_clean_stop.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
data_lemma=dataset[['class','text_lemma'] ]

data_lemma.to_csv("text_lemma.csv", index=False)
files.download('text_lemma.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [47]:
data_tokenize=dataset[['class','text_tokenize'] ]

data_tokenize.to_csv("text_tokenize.csv", index=False)
files.download('text_tokenize.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle 
# Save the model tf-idf
with open('tfidf.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)  